In [1]:
from kymatio.torch import TimeFrequencyScattering
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import sys
import torch
import tqdm

import cnn
import synth

# duration = 4
# event_duration = 2**(-4)
# sr = 2**13
# fmin = 2**8
# fmax = 2**11
# Q = 24
# hop_length = 2**6
# n_events = 2**6

n_densities = 7
n_slopes = 1
n_folds = n_densities * n_slopes
batch_size = 1

dataset = cnn.ChirpTextureDataModule(
    n_densities=n_densities,
    n_slopes=n_slopes,
    n_folds=n_folds,
    batch_size=batch_size)
dataset.setup()

def scattering_audio_distance(x, y, S2y, sc):
    Sx = sc(x)
    meta = sc.meta()
    S2x = torch.stack([Sx[key] for key, order in zip(meta['key'], meta['order']) if order==2]).squeeze()
    # inspired by Engel et al. ICLR 2020
    eps = 1e-5
    lin_distance = torch.linalg.vector_norm(S2x - S2y, ord=1)
    log_distance = torch.linalg.vector_norm(torch.log(eps+S2x) - torch.log(eps+S2y), ord=1)
    return lin_distance + log_distance

CQT kernels created, time used = 0.0224 seconds
CQT kernels created, time used = 0.0233 seconds
CQT kernels created, time used = 0.0244 seconds


In [2]:
sc = TimeFrequencyScattering(
    shape=(2**15),
    J=6,
    Q=(24, 2),
    Q_fr=2,
    J_fr=6,
    T='global',
    F='global',
    format='time',
    out_type='dict'
)

y = synth.generate_chirp_texture(
    theta_density=torch.tensor(0.5),
    theta_slope=torch.tensor(0.),
    duration=dataset.train_ds.duration,
    event_duration=dataset.train_ds.event_duration,
    sr=dataset.train_ds.sr,
    fmin=dataset.train_ds.fmin,
    fmax=dataset.train_ds.fmax,
    n_events=dataset.train_ds.n_events,
    Q=dataset.train_ds.Q,
    hop_length=dataset.train_ds.hop_length,
    seed=0,
)

Sy = sc(y)
meta = sc.meta()
S2y = torch.stack([Sy[key] for key, order in zip(meta['key'], meta['order']) if order==2]).squeeze()

In [3]:
distance = lambda x, y: scattering_audio_distance(x, y, S2y, sc)
seed = 1

sp_df = pd.DataFrame(columns=[
    "density_idx", "slope_idx", "distance", "density_grad", "slope_grad"])

for _, row in tqdm.tqdm(dataset.df.iterrows()):
    theta_density = torch.tensor(row["density"], requires_grad=False)
    theta_slope = torch.tensor(row["slope"], requires_grad=False)
    x = synth.generate_chirp_texture(
        theta_density=theta_density,
        theta_slope=theta_slope,
        duration=dataset.train_ds.duration,
        event_duration=dataset.train_ds.event_duration,
        sr=dataset.train_ds.sr,
        fmin=dataset.train_ds.fmin,
        fmax=dataset.train_ds.fmax,
        n_events=dataset.train_ds.n_events,
        Q=dataset.train_ds.Q,
        hop_length=dataset.train_ds.hop_length,
        seed=seed,
    )
    xy_dist = distance(x, y)
    print(xy_dist)
    xy_dist.backward()
    if theta_density.grad is None:
        theta_density.grad = torch.zeros_like(theta_density)
    if theta_slope.grad is None:
        theta_slope.grad = torch.zeros_like(theta_slope)
    sp_row = {
        "density_idx": int(row['density_idx']),
        "slope_idx": int(row['slope_idx']),
        "distance": xy_dist.item(),
        "density_grad": -theta_density.grad.item(),
        "slope_grad": -theta_slope.grad.item(),
    }
    sp_df.loc[len(sp_df)] = sp_row

0it [00:37, ?it/s]


TypeError: linalg_norm() received an invalid combination of arguments - got (Tensor, p=float), but expected one of:
 * (Tensor input, Number ord, tuple of ints dim, bool keepdim, *, torch.dtype dtype, Tensor out)
 * (Tensor input, str ord, tuple of ints dim, bool keepdim, *, torch.dtype dtype, Tensor out)
